In [ ]:
import helper

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

In [3]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

In [2]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
   
    source_id_text = []
    target_id_text = []
    for text in source_text.split('\n'):
        t = []
        for word in text.split():
            t.append(source_vocab_to_int[word])
        source_id_text.append(t)
    for text in target_text.split('\n'):
        t = []
        for word in text.split():
            t.append(target_vocab_to_int[word])
        t.append(target_vocab_to_int['<EOS>'])
        target_id_text.append(t)
    return source_id_text, target_id_text

In [6]:
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

In [7]:

import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

In [3]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

In [4]:
def model_inputs():
    inputs = tf.placeholder(tf.int32,[None,None],name='input')
    targets = tf.placeholder(tf.int32,[None,None])
    learning_rate = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    return inputs, targets, learning_rate, keep_prob

In [5]:
def process_decoding_input(target_data, target_vocab_to_int, batch_size):
    x = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1,1])
    y = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), x], 1)
       
    return y

In [7]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm,output_keep_prob=keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([drop]*num_layers)
    _, encoder_state = tf.nn.dynamic_rnn(cell,rnn_inputs,dtype=tf.float32)
    return encoder_state

In [8]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob):
    decoder_fn_train = tf.contrib.seq2seq.simple_decoder_fn_train(encoder_state)
    output,_,_ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell,
                    decoder_fn_train,
                    dec_embed_input,
                    sequence_length,
                    scope=decoding_scope,
                )
    logits = output_fn(output)
    logits = tf.nn.dropout(logits,keep_prob)
    return logits

In [9]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob):
   

    dynamic_fn_inference = tf.contrib.seq2seq.simple_decoder_fn_inference(output_fn, 
                                                                          encoder_state,
                                                                          dec_embeddings,
                                                                          start_of_sequence_id,
                                                                         end_of_sequence_id,
                                                                          maximum_length-1,
                                                                         vocab_size)
    outputs_inference,_,_ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell,dynamic_fn_inference,
                                                                           scope=decoding_scope)

#     logits = tf.nn.dropout(outputs_inference,keep_prob=keep_prob)

    return outputs_inference



In [10]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, target_vocab_to_int, keep_prob):
    
    start_of_sequence_id = target_vocab_to_int['<GO>']
    end_of_sequence_id = target_vocab_to_int['<EOS>']
    
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    dec_cell = tf.contrib.rnn.MultiRNNCell([drop]*num_layers)
    
    with tf.variable_scope('decoding') as decoding_scope:
        output_fn = lambda x: tf.contrib.layers.fully_connected(x,
                                                               vocab_size,
                                                               None,
                                                               scope=decoding_scope)
    with tf.variable_scope('decoding') as decoding_scope:
        training_logits = decoding_layer_train(encoder_state,
                                              dec_cell,
                                              dec_embed_input,
                                              sequence_length,
                                              decoding_scope,
                                              output_fn,
                                              keep_prob)
    with tf.variable_scope('decoding', reuse=True) as decoding_scope:
        inference_logits = decoding_layer_infer(encoder_state,
                                               dec_cell,
                                               dec_embeddings,
                                               start_of_sequence_id,
                                               end_of_sequence_id,
                                               sequence_length,
                                               vocab_size,
                                               decoding_scope,
                                               output_fn,
                                               keep_prob)
    
    return (training_logits, inference_logits)



In [11]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, rnn_size, num_layers, target_vocab_to_int):
   
    enc_embeddings = tf.Variable(tf.truncated_normal([source_vocab_size,enc_embedding_size],stddev=0.1))
    dec_embeddings = tf.Variable(tf.truncated_normal([target_vocab_size,dec_embedding_size],stddev=0.1))
#     rnn_inputs = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, enc_embedding_size)
    rnn_inputs = tf.nn.embedding_lookup(enc_embeddings,input_data)
    
    dec_input = process_decoding_input(target_data,target_vocab_to_int,batch_size)
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings,dec_input)
    
    encoder_state = encoding_layer(rnn_inputs,rnn_size,num_layers,keep_prob)
    
    training_logits, inference_logits = decoding_layer(dec_embed_input,dec_embeddings,encoder_state,target_vocab_size,sequence_length
                                                      ,rnn_size,num_layers,target_vocab_to_int,keep_prob)
    
    return (training_logits, inference_logits)


In [21]:
# Number of Epochs
epochs = 8
# Batch Size
batch_size = 275
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 256
decoding_embedding_size = 256
# Learning Rate
learning_rate = 0.005
# Dropout Keep Probability
keep_probability = 0.9

In [22]:
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_source_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob = model_inputs()
    sequence_length = tf.placeholder_with_default(max_source_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)
    
    train_logits, inference_logits = seq2seq_model(
        tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(source_vocab_to_int), len(target_vocab_to_int),
        encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, target_vocab_to_int)

    tf.identity(inference_logits, 'logits')
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            train_logits,
            targets,
            tf.ones([input_shape[0], sequence_length]))

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

In [23]:

import time

def get_accuracy(target, logits):

    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1]), (0,0)],
            'constant')

    return np.mean(np.equal(target, np.argmax(logits, 2)))

train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]

valid_source = helper.pad_sentence_batch(source_int_text[:batch_size])
valid_target = helper.pad_sentence_batch(target_int_text[:batch_size])

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch) in enumerate(
                helper.batch_data(train_source, train_target, batch_size)):
            start_time = time.time()
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 sequence_length: target_batch.shape[1],
                 keep_prob: keep_probability})
            
            batch_train_logits = sess.run(
                inference_logits,
                {input_data: source_batch, keep_prob: 1.0})
            batch_valid_logits = sess.run(
                inference_logits,
                {input_data: valid_source, keep_prob: 1.0})
                
            train_acc = get_accuracy(target_batch, batch_train_logits)
            valid_acc = get_accuracy(np.array(valid_target), batch_valid_logits)
            end_time = time.time()
            if batch_i % 50 == 0:
                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.3f}, Validation Accuracy: {:>6.3f}, Loss: {:>6.3f}'
                  .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    0/501 - Train Accuracy:  0.274, Validation Accuracy:  0.346, Loss:  5.878
Epoch   0 Batch   50/501 - Train Accuracy:  0.413, Validation Accuracy:  0.466, Loss:  1.887
Epoch   0 Batch  100/501 - Train Accuracy:  0.514, Validation Accuracy:  0.545, Loss:  1.204
Epoch   0 Batch  150/501 - Train Accuracy:  0.614, Validation Accuracy:  0.602, Loss:  1.009
Epoch   0 Batch  200/501 - Train Accuracy:  0.617, Validation Accuracy:  0.632, Loss:  0.951
Epoch   0 Batch  250/501 - Train Accuracy:  0.689, Validation Accuracy:  0.691, Loss:  0.859
Epoch   0 Batch  300/501 - Train Accuracy:  0.728, Validation Accuracy:  0.727, Loss:  0.784
Epoch   0 Batch  350/501 - Train Accuracy:  0.775, Validation Accuracy:  0.755, Loss:  0.726
Epoch   0 Batch  400/501 - Train Accuracy:  0.811, Validation Accuracy:  0.805, Loss:  0.647
Epoch   0 Batch  450/501 - Train Accuracy:  0.828, Validation Accuracy:  0.847, Loss:  0.570
Epoch   1 Batch    0/501 - Train Accuracy:  0.903, Validation Accuracy

In [24]:
helper.save_params(save_path)

In [25]:
import tensorflow as tf
import numpy as np
import helper
_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

In [26]:
def sentence_to_seq(sentence, vocab_to_int):
    sentence = sentence.lower()
    seq = []
    for word in sentence.split():
        if word not in vocab_to_int.keys():
            seq.append(vocab_to_int['<UNK>'])
        else:
            seq.append(vocab_to_int[word])
    return seq

Tests Passed


In [27]:
translate_sentence = 'he saw a old yellow truck .'


translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('logits:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence], keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in np.argmax(translate_logits, 1)]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)]))

Input
  Word Ids:      [105, 94, 133, 32, 163, 80, 45]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [254, 135, 92, 158, 116, 94, 124, 267, 1]
  French Words: ['il', 'a', 'vu', 'un', 'vieux', 'camion', 'jaune', '.', '<EOS>']
